In [69]:
import pandas as pd
import numpy as np # type: ignore
from pybaseball import *
import statsapi
import os

from pull_data import *
%load_ext autoreload
%autoreload 2
import requests

from pybaseball import cache
cache.enable()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [70]:
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

team_ids = [138, 140, 145, 110, 111, 144, 158, 143, 133, 141, 108, 114, 115, 121, 137, 146, 116, 118, 139, 117, 113, 112, 142, 136, 120, 109, 134, 135, 147, 119]

In [71]:
# Update roster player ids
reload = True
if reload:
    player_id_mapping = map_player_ids(team_ids)
else:
    player_id_mapping = pd.read_csv('player_id_mapping.csv')


No identically matched names found! Returning the 5 most similar names.
Victor Scott
Luis Robert
Michael Taylor
No identically matched names found! Returning the 5 most similar names.
Enyel De Los Santos
Michael Harris
Ronald Acuña
No identically matched names found! Returning the 5 most similar names.
Vladimir Guerrero
Luis L. Ortiz
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
Jung Hoo Lee
No identically matched names found! Returning the 5 most similar names.
LaMonte Wade
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
Bobby Witt
Daniel Lynch
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Re

In [72]:
player_id_mapping[player_id_mapping['team'] == 'White Sox']

,name,abbrev,clean_name,clean_abbrev,team,player_id
52,Adrian Houser,A. Houser,Adrian Houser,A. Houser,White Sox,605288
53,Andrew Benintendi,A. Benintendi,Andrew Benintendi,A. Benintendi,White Sox,643217
54,Austin Slater,A. Slater,Austin Slater,A. Slater,White Sox,596103
55,Brandon Eisert,B. Eisert,Brandon Eisert,B. Eisert,White Sox,685126
56,Bryse Wilson,B. Wilson,Bryse Wilson,B. Wilson,White Sox,669060
57,Caleb Freeman,C. Freeman,Caleb Freeman,C. Freeman,White Sox,673965
58,Cam Booser,C. Booser,Cam Booser,C. Booser,White Sox,592155
59,Chase Meidroth,C. Meidroth,Chase Meidroth,C. Meidroth,White Sox,805367
60,Dan Altavilla,D. Altavilla,Dan Altavilla,D. Altavilla,White Sox,656186
61,Davis Martin,D. Martin,Davis Martin,D. Martin,White Sox,663436


In [73]:
from datetime import date

# Get today's date in YYYY-MM-DD format
days_offset = 0
today = (date.today() + pd.DateOffset(days=days_offset)).strftime('%Y-%m-%d')

# Get today's schedule
games = statsapi.schedule(start_date=today, end_date=today)

# Loop through games and extract probable pitchers
for game in games:
    home_team = game['home_name']
    away_team = game['away_name']
    home_pitcher = game.get('home_probable_pitcher', 'TBD')
    away_pitcher = game.get('away_probable_pitcher', 'TBD')
    
    print(f"{away_team} ({away_pitcher}) @ {home_team} ({home_pitcher})")

Houston Astros (Lance McCullers Jr.) @ Pittsburgh Pirates (Paul Skenes)
Colorado Rockies (Chase Dollander) @ Miami Marlins (Sandy Alcantara)
Chicago Cubs (Cade Horton) @ Washington Nationals (Trevor Williams)
Cleveland Guardians (Tanner Bibee) @ New York Yankees (Carlos Rodón)
Philadelphia Phillies (Cristopher Sánchez) @ Toronto Blue Jays (Bowden Francis)
Los Angeles Angels (Yusei Kikuchi) @ Boston Red Sox (Brayan Bello)
Milwaukee Brewers (Freddy Peralta) @ Cincinnati Reds (Hunter Greene)
Arizona Diamondbacks (Zac Gallen) @ Atlanta Braves (Spencer Strider)
Texas Rangers (Tyler Mahle) @ Tampa Bay Rays (Drew Rasmussen)
Detroit Tigers (Beau Brieske) @ Chicago White Sox (Shane Smith)
Kansas City Royals (Michael Lorenzen) @ St. Louis Cardinals (Andre Pallante)
Baltimore Orioles (Tomoyuki Sugano) @ Seattle Mariners (George Kirby)
San Diego Padres (Ryan Bergert) @ San Francisco Giants (Landen Roupp)
Minnesota Twins (Pablo López) @ Athletics (Grant Holman)
New York Mets (Tylor Megill) @ Los An

In [74]:
pitching_df, batting_df = get_rotowire_lineups()

In [75]:
def spray_angle(hc_x, hc_y, stand):
    mult = 1
    if stand == 'L':
        mult = -1
    return mult * np.degrees(np.arctan2((hc_x - 125.42), (198.27 - hc_y)))

In [76]:
import os
api_key = os.getenv("api_key")
print(api_key)

fa1d935c1d67073458a1f970bc434075


In [77]:
# Note that BBE = attempts = number of events
pitcher_barrels = statcast_pitcher_exitvelo_barrels(2025)

In [78]:
start_date = (date.today() + pd.DateOffset(days=-30)).strftime('%Y-%m-%d')
end_date = today
pitcher_ids = get_starting_ids(days_offset=days_offset)
pitcher_id_nos = [p[0] for p in pitcher_ids]
starters_data = {}
pitchers_arsenals = {}
pitcher_opponents = {}
pitcher_games = {}
for id, opposing_team, game_id in pitcher_ids:
    pitcher_data = statcast_pitcher(start_dt=start_date, end_dt=end_date,player_id=id)
    pitcher_opponents[id] = opposing_team
    starters_data[id] = pitcher_data.copy()
    pitcher_games[id] = game_id
    pitchers_arsenals[id] = {'R': get_pitch_arsenal(pitcher_data, 'R'), 'L': get_pitch_arsenal(pitcher_data, 'L')}

['Paul Skenes', 'Lance McCullers Jr.', 'Sandy Alcantara', 'Chase Dollander', 'Trevor Williams', 'Cade Horton', 'Carlos Rodón', 'Tanner Bibee', 'Bowden Francis', 'Cristopher Sánchez', 'Brayan Bello', 'Yusei Kikuchi', 'Hunter Greene', 'Freddy Peralta', 'Spencer Strider', 'Zac Gallen', 'Drew Rasmussen', 'Tyler Mahle', 'Shane Smith', 'Beau Brieske', 'Andre Pallante', 'Michael Lorenzen', 'George Kirby', 'Tomoyuki Sugano', 'Landen Roupp', 'Ryan Bergert', 'Grant Holman', 'Pablo López', 'Clayton Kershaw', 'Tylor Megill']
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gatheri

In [79]:
# pitcher_barrels_copy = pitcher_barrels[pitcher_barrels['attempts'] > 50]
pitcher_barrels_copy = pitcher_barrels[pitcher_barrels['player_id'].isin(pitcher_id_nos)].copy()
pitcher_barrels_copy['barrel_percentage'] = pitcher_barrels_copy['player_id'].apply(get_barrel_percentage, start_date=start_date, end_date=end_date)
pitcher_barrels_copy['left_barrel_pct'] = pitcher_barrels_copy['barrel_percentage'].apply(lambda x: x['left_pct'])
pitcher_barrels_copy['right_barrel_pct'] = pitcher_barrels_copy['barrel_percentage'].apply(lambda x: x['right_pct'])
pitcher_barrels_copy['opponent'] = pitcher_barrels_copy['player_id'].apply(lambda x: pitcher_opponents[x])
pitcher_barrels_copy['game_id'] = pitcher_barrels_copy['player_id'].apply(lambda x: pitcher_games.get(x))
pitcher_barrels_copy[['last_name, first_name','player_id','game_id','attempts','barrels','brl_percent','barrel_percentage', 'left_barrel_pct', 'right_barrel_pct', 'opponent']]

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data


,"last_name, first_name",player_id,game_id,attempts,barrels,brl_percent,barrel_percentage,left_barrel_pct,right_barrel_pct,opponent
16,"Sugano, Tomoyuki",608372,777655,209,21,10.00,"{'left_barrels': 4, 'right_barrels': 3, 'total...",0.11,0.06,home
33,"Pallante, Andre",669467,777660,199,16,8.00,"{'left_barrels': 6, 'right_barrels': 3, 'total...",0.12,0.07,away
34,"Lorenzen, Michael",547179,777660,199,17,8.50,"{'left_barrels': 4, 'right_barrels': 5, 'total...",0.10,0.09,home
36,"Skenes, Paul",694973,777663,196,9,4.60,"{'left_barrels': 4, 'right_barrels': 0, 'total...",0.09,0.00,away
44,"Gallen, Zac",668678,777672,192,19,9.90,"{'left_barrels': 2, 'right_barrels': 6, 'total...",0.05,0.14,home
51,"Kikuchi, Yusei",579328,777667,190,16,8.40,"{'left_barrels': 1, 'right_barrels': 3, 'total...",0.04,0.06,home
53,"Bibee, Tanner",676440,777664,188,18,9.60,"{'left_barrels': 3, 'right_barrels': 4, 'total...",0.06,0.09,home
54,"Mahle, Tyler",641816,777662,187,13,7.00,"{'left_barrels': 3, 'right_barrels': 5, 'total...",0.09,0.09,home
58,"Williams, Trevor",592866,777659,184,11,6.00,"{'left_barrels': 3, 'right_barrels': 2, 'total...",0.07,0.09,away
67,"Peralta, Freddy",642547,777668,176,15,8.50,"{'left_barrels': 3, 'right_barrels': 4, 'total...",0.07,0.14,home


In [80]:
filtered_pitcher_barrels_copy = pitcher_barrels_copy[(pitcher_barrels_copy['left_barrel_pct'] > 0.07) | (pitcher_barrels_copy['right_barrel_pct'] > 0.07)].copy()
filtered_pitcher_barrels_copy['sides'] = filtered_pitcher_barrels_copy.apply(lambda row: get_sides(row['left_barrel_pct'], row['right_barrel_pct']),axis=1)
filtered_pitcher_barrels_copy['arsenal'] = filtered_pitcher_barrels_copy.apply(lambda row: pitchers_arsenals[row['player_id']], axis=1)
filtered_pitcher_barrels_copy[['last_name, first_name','player_id','game_id','attempts','barrels','brl_percent','barrel_percentage', 'left_barrel_pct', 'right_barrel_pct', 'sides', 'arsenal', 'opponent']]

,"last_name, first_name",player_id,game_id,attempts,barrels,brl_percent,barrel_percentage,left_barrel_pct,right_barrel_pct,sides,arsenal,opponent
16,"Sugano, Tomoyuki",608372,777655,209,21,10.00,"{'left_barrels': 4, 'right_barrels': 3, 'total...",0.11,0.06,[L],"{'R': {'ST': 32.38095238095238, 'FC': 19.04761...",home
33,"Pallante, Andre",669467,777660,199,16,8.00,"{'left_barrels': 6, 'right_barrels': 3, 'total...",0.12,0.07,[L],"{'R': {'SI': 32.460732984293195, 'FF': 29.3193...",away
34,"Lorenzen, Michael",547179,777660,199,17,8.50,"{'left_barrels': 4, 'right_barrels': 5, 'total...",0.10,0.09,"[L, R]","{'R': {'SL': 26.0, 'FF': 20.8}, 'L': {'FF': 33...",home
36,"Skenes, Paul",694973,777663,196,9,4.60,"{'left_barrels': 4, 'right_barrels': 0, 'total...",0.09,0.00,[L],"{'R': {'FF': 33.624454148471614, 'ST': 24.8908...",away
44,"Gallen, Zac",668678,777672,192,19,9.90,"{'left_barrels': 2, 'right_barrels': 6, 'total...",0.05,0.14,[R],"{'R': {'FF': 51.46443514644351, 'KC': 20.50209...",home
53,"Bibee, Tanner",676440,777664,188,18,9.60,"{'left_barrels': 3, 'right_barrels': 4, 'total...",0.06,0.09,[R],"{'R': {'FF': 24.271844660194176, 'FC': 23.7864...",home
54,"Mahle, Tyler",641816,777662,187,13,7.00,"{'left_barrels': 3, 'right_barrels': 5, 'total...",0.09,0.09,"[L, R]","{'R': {'FF': 48.72727272727273, 'SL': 20.72727...",home
58,"Williams, Trevor",592866,777659,184,11,6.00,"{'left_barrels': 3, 'right_barrels': 2, 'total...",0.07,0.09,[R],"{'R': {'FF': 40.140845070422536, 'ST': 33.8028...",away
67,"Peralta, Freddy",642547,777668,176,15,8.50,"{'left_barrels': 3, 'right_barrels': 4, 'total...",0.07,0.14,[R],"{'R': {'FF': 62.359550561797754}, 'L': {'FF': ...",home
73,"Roupp, Landen",694738,777656,171,12,7.00,"{'left_barrels': 7, 'right_barrels': 1, 'total...",0.14,0.03,[L],"{'R': {'CU': 45.78313253012048, 'SI': 40.96385...",away


Next step is
- For each batter in a lineup that a pitcher is facing, we want to get their barrel % against the pitcher's arsenal (the pitchers > 18%)
- Calculate the weighted pull air percentage (pull % * air %)
- Also eventually can add blast %, iso

In [81]:
player_id_mapping[player_id_mapping['team'] == 'Padres']

,name,abbrev,clean_name,clean_abbrev,team,player_id
702,Adrian Morejon,A. Morejon,Adrian Morejon,A. Morejon,Padres,670970
703,Brandon Lockridge,B. Lockridge,Brandon Lockridge,B. Lockridge,Padres,663604
704,David Morgan,D. Morgan,David Morgan,D. Morgan,Padres,688158
705,Dylan Cease,D. Cease,Dylan Cease,D. Cease,Padres,656302
706,Elias Díaz,E. Díaz,Elias Diaz,E. Diaz,Padres,553869
707,Fernando Tatis,F. Tatis,Fernando Tatis,F. Tatis,Padres,665487
708,Gavin Sheets,G. Sheets,Gavin Sheets,G. Sheets,Padres,657757
709,Jackson Merrill,J. Merrill,Jackson Merrill,J. Merrill,Padres,701538
710,Jake Cronenworth,J. Cronenworth,Jake Cronenworth,J. Cronenworth,Padres,630105
711,Jason Adam,J. Adam,Jason Adam,J. Adam,Padres,592094


In [82]:
pitcher_df = filtered_pitcher_barrels_copy.copy()
pitcher_df['hitter_team'] = pitcher_df.apply(lambda row: get_team(row['game_id'], row['opponent']), axis=1)
pitcher_df['opposing_lineup'] = pitcher_df.apply(lambda row: get_opposing_lineup(row['game_id'], row['opponent']), axis=1)
pitcher_df['opposing_lineup_names'] = pitcher_df.apply(lambda row: get_specific_lineup(row['hitter_team'], batting_df), axis=1)
pitcher_df['rotowire_ids'] = pitcher_df.apply(lambda row: get_player_ids(row['opposing_lineup_names'], player_id_mapping, row['hitter_team']), axis=1)
pitcher_df[['last_name, first_name','player_id','game_id','attempts','barrels','brl_percent','barrel_percentage', 'left_barrel_pct', 'right_barrel_pct', 'sides', 'arsenal', 'opponent', 'opposing_lineup', 'opposing_lineup_names','hitter_team','rotowire_ids']]

,"last_name, first_name",player_id,game_id,attempts,barrels,brl_percent,barrel_percentage,left_barrel_pct,right_barrel_pct,sides,arsenal,opponent,opposing_lineup,opposing_lineup_names,hitter_team,rotowire_ids
16,"Sugano, Tomoyuki",608372,777655,209,21,10.00,"{'left_barrels': 4, 'right_barrels': 3, 'total...",0.11,0.06,[L],"{'R': {'ST': 32.38095238095238, 'FC': 19.04761...",home,"[641487, 593871, 677594, 663728, 668227, 64213...","[J. Crawford, J. Polanco, J. Rodriguez, Cal Ra...",Mariners,"[641487, 593871, 677594, 663728, 668227, 64213..."
33,"Pallante, Andre",669467,777660,199,16,8.00,"{'left_barrels': 6, 'right_barrels': 3, 'total...",0.12,0.07,[L],"{'R': {'SI': 32.460732984293195, 'FF': 29.3193...",away,"[663697, 677951, 672580, 686469, 521692, 69550...","[J. India, Bobby Witt, M. Garcia, V. Pasquanti...",Royals,"[663697, 677951, 672580, 686469, 521692, 11485..."
34,"Lorenzen, Michael",547179,777660,199,17,8.50,"{'left_barrels': 4, 'right_barrels': 5, 'total...",0.10,0.09,"[L, R]","{'R': {'SL': 26.0, 'FF': 20.8}, 'L': {'FF': 33...",home,"[663457, 691026, 680977, 575929, 671056, 57144...","[L. Nootbaar, Masyn Winn, B. Donovan, W. Contr...",Cardinals,"[663457, 691026, 680977, 575929, 671056, 57144..."
36,"Skenes, Paul",694973,777663,196,9,4.60,"{'left_barrels': 4, 'right_barrels': 0, 'total...",0.09,0.00,[L],"{'R': {'FF': 33.624454148471614, 'ST': 24.8908...",away,"[665161, 670623, 514888, 572233, 673237, 60517...","[Jeremy Pena, I. Paredes, Jose Altuve, C. Walk...",Astros,"[665161, 670623, 514888, 572233, 673237, 60517..."
44,"Gallen, Zac",668678,777672,192,19,9.90,"{'left_barrels': 2, 'right_barrels': 6, 'total...",0.05,0.14,[R],"{'R': {'FF': 51.46443514644351, 'KC': 20.50209...",home,"[660670, 686948, 663586, 621566, 542303, 64527...","[Ronald Acuna, D. Baldwin, Austin Riley, Matt ...",Braves,"[660670, 686948, 663586, 621566, 542303, 64527..."
53,"Bibee, Tanner",676440,777664,188,18,9.60,"{'left_barrels': 3, 'right_barrels': 4, 'total...",0.06,0.09,[R],"{'R': {'FF': 24.271844660194176, 'FC': 23.7864...",home,"[663757, 700250, 592450, 641355, 502671, 66586...","[T. Grisham, Ben Rice, Aaron Judge, C. Belling...",Yankees,"[663757, 700250, 592450, 641355, 502671, 66586..."
54,"Mahle, Tyler",641816,777662,187,13,7.00,"{'left_barrels': 3, 'right_barrels': 5, 'total...",0.09,0.09,"[L, R]","{'R': {'FF': 48.72727272727273, 'SL': 20.72727...",home,"[666139, 664040, 650490, 666018, 691406, 66396...","[Josh Lowe, Brandon Lowe, Yandy Diaz, J. Arand...",Rays,"[666139, 664040, 650490, 666018, 691406, 66396..."
58,"Williams, Trevor",592866,777659,184,11,6.00,"{'left_barrels': 3, 'right_barrels': 2, 'total...",0.07,0.09,[R],"{'R': {'FF': 40.140845070422536, 'ST': 33.8028...",away,"[664023, 673548, 691718, 608348, 683737, 62102...","[Ian Happ, Seiya Suzuki, P. Crow-Armstrong, Ca...",Cubs,"[664023, 673548, 691718, 608348, 683737, 62102..."
67,"Peralta, Freddy",642547,777668,176,15,8.50,"{'left_barrels': 3, 'right_barrels': 4, 'total...",0.07,0.14,[R],"{'R': {'FF': 62.359550561797754}, 'L': {'FF': ...",home,"[670770, 666158, 656582, 682829, 663886, 66871...","[TJ Friedl, Gavin Lux, E. De La Cruz, T. Steph...",Reds,"[670770, 666158, 682829, 663886, 668715, 66618..."
73,"Roupp, Landen",694738,777656,171,12,7.00,"{'left_barrels': 7, 'right_barrels': 1, 'total...",0.14,0.03,[L],"{'R': {'CU': 45.78313253012048, 'SI': 40.96385...",away,"[665487, 650333, 592518, 701538, 657757, 59342...","[F. Tatis, Luis Arraez, M. Machado, J. Merrill...",Padres,"[665487, 650333, 592518, 701538, 657757, 59342..."


In [83]:
player_id_mapping[player_id_mapping['team'] == 'Mariners']

,name,abbrev,clean_name,clean_abbrev,team,player_id
598,Andrés Muñoz,A. Muñoz,Andres Munoz,A. Munoz,Mariners,662253
599,Ben Williamson,B. Williamson,Ben Williamson,B. Williamson,Mariners,810938
600,Bryan Woo,B. Woo,Bryan Woo,B. Woo,Mariners,693433
601,Bryce Miller,B. Miller,Bryce Miller,B. Miller,Mariners,682243
602,Cal Raleigh,C. Raleigh,Cal Raleigh,C. Raleigh,Mariners,663728
603,Carlos Vargas,C. Vargas,Carlos Vargas,C. Vargas,Mariners,672841
604,Casey Lawrence,C. Lawrence,Casey Lawrence,C. Lawrence,Mariners,596271
605,Cole Young,C. Young,Cole Young,C. Young,Mariners,702284
606,Collin Snider,C. Snider,Collin Snider,C. Snider,Mariners,676092
607,Donovan Solano,D. Solano,Donovan Solano,D. Solano,Mariners,456781


In [84]:
hitter_barrels = statcast_batter_exitvelo_barrels(2025)

In [85]:
def get_hitter_barrels(hitter_barrels, hitter_ids, start_date, end_date, pitch_mix, sides):
    if hitter_ids is None or len(hitter_ids) == 0:
        return {}
    hitter_barrels_copy = hitter_barrels[hitter_barrels['player_id'].isin(hitter_ids)].copy()
    hitter_barrels_copy['barrel_data'] = hitter_barrels_copy['player_id'].apply(hitter_barrel_percentage, start_date=start_date, end_date=end_date, pitch_mix=pitch_mix, sides=sides)
    hitter_barrels_copy['barrel_percentage'] = hitter_barrels_copy.apply(lambda row: get_barrel_pcts(row['barrel_data']), axis=1)
    hitter_barrels_copy['barrel_qual'] = hitter_barrels_copy['barrel_percentage'].apply(lambda x: check_barrel_pcts(x))
    # print(hitter_barrels_copy['barrel_qual'])
    filtered = hitter_barrels_copy[hitter_barrels_copy['barrel_qual'].apply(lambda x: x != [0,0])].copy()
    # print(filtered)
    if len(filtered) > 3:
        # Create a new column with just the first element
        filtered['barrel_qual_first'] = filtered['barrel_qual'].apply(lambda x: x[0])
        # Sort by that column and get top 3
        top_3 = filtered.nlargest(3, 'barrel_qual_first')
        # Drop the temporary column if you don't need it
        top_3 = top_3.drop('barrel_qual_first', axis=1)
        return dict(zip(top_3['last_name, first_name'], top_3['barrel_qual']))
    else:
        return dict(zip(filtered['last_name, first_name'], filtered['barrel_qual']))

In [88]:
hitter_pitcher_df = pitcher_df.copy()
hitter_pitcher_df['hitter_barrels'] = hitter_pitcher_df.apply(lambda row: get_hitter_barrels(hitter_barrels, row['rotowire_ids'], start_date, end_date, row['arsenal'], row['sides']), axis=1)
hitter_pitcher_df['num_hitters'] = hitter_pitcher_df['hitter_barrels'].apply(lambda x: len(x))
hitter_pitcher_df['hitter_team'] = hitter_pitcher_df.apply(lambda row: get_team(row['game_id'], row['opponent']), axis=1)
hitter_pitcher_df['start_time'] = hitter_pitcher_df['game_id'].apply(lambda x: get_start_time(x))
hitter_pitcher_df[['last_name, first_name','player_id','game_id','attempts','barrels','brl_percent','barrel_percentage', 'left_barrel_pct', 'right_barrel_pct', 'sides', 'arsenal', 'opponent', 'opposing_lineup', 'hitter_barrels', 'num_hitters', 'hitter_team', 'start_time']]

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

,"last_name, first_name",player_id,game_id,attempts,barrels,brl_percent,barrel_percentage,left_barrel_pct,right_barrel_pct,sides,arsenal,opponent,opposing_lineup,hitter_barrels,num_hitters,hitter_team,start_time
16,"Sugano, Tomoyuki",608372,777655,209,21,10.00,"{'left_barrels': 4, 'right_barrels': 3, 'total...",0.11,0.06,[L],"{'R': {'ST': 32.38095238095238, 'FC': 19.04761...",home,"[641487, 593871, 677594, 663728, 668227, 64213...","{'Raleigh, Cal': [0.24390243902439024, 0.41463...",2,Mariners,6:41 PM.
33,"Pallante, Andre",669467,777660,199,16,8.00,"{'left_barrels': 6, 'right_barrels': 3, 'total...",0.12,0.07,[L],"{'R': {'SI': 32.460732984293195, 'FF': 29.3193...",away,"[663697, 677951, 672580, 686469, 521692, 69550...",{},0,Royals,6:45 PM.
34,"Lorenzen, Michael",547179,777660,199,17,8.50,"{'left_barrels': 4, 'right_barrels': 5, 'total...",0.10,0.09,"[L, R]","{'R': {'SL': 26.0, 'FF': 20.8}, 'L': {'FF': 33...",home,"[663457, 691026, 680977, 575929, 671056, 57144...","{'Contreras, Willson': [0.140625, 0.375]}",1,Cardinals,6:45 PM.
36,"Skenes, Paul",694973,777663,196,9,4.60,"{'left_barrels': 4, 'right_barrels': 0, 'total...",0.09,0.00,[L],"{'R': {'FF': 33.624454148471614, 'ST': 24.8908...",away,"[665161, 670623, 514888, 572233, 673237, 60517...",{},0,Astros,"15,891."
44,"Gallen, Zac",668678,777672,192,19,9.90,"{'left_barrels': 2, 'right_barrels': 6, 'total...",0.05,0.14,[R],"{'R': {'FF': 51.46443514644351, 'KC': 20.50209...",home,"[660670, 686948, 663586, 621566, 542303, 64527...","{'Riley, Austin': [0.14285714285714285, 0.3896...",2,Braves,"32,898."
53,"Bibee, Tanner",676440,777664,188,18,9.60,"{'left_barrels': 3, 'right_barrels': 4, 'total...",0.06,0.09,[R],"{'R': {'FF': 24.271844660194176, 'FC': 23.7864...",home,"[663757, 700250, 592450, 641355, 502671, 66586...","{'Judge, Aaron': [0.2318840579710145, 0.463768...",1,Yankees,"40,683."
54,"Mahle, Tyler",641816,777662,187,13,7.00,"{'left_barrels': 3, 'right_barrels': 5, 'total...",0.09,0.09,"[L, R]","{'R': {'FF': 48.72727272727273, 'SL': 20.72727...",home,"[666139, 664040, 650490, 666018, 691406, 66396...","{'Díaz, Yandy': [0.11688311688311688, 0.454545...",3,Rays,"9,131."
58,"Williams, Trevor",592866,777659,184,11,6.00,"{'left_barrels': 3, 'right_barrels': 2, 'total...",0.07,0.09,[R],"{'R': {'FF': 40.140845070422536, 'ST': 33.8028...",away,"[664023, 673548, 691718, 608348, 683737, 62102...","{'Swanson, Dansby': [0.11594202898550725, 0.27...",2,Cubs,"27,702."
67,"Peralta, Freddy",642547,777668,176,15,8.50,"{'left_barrels': 3, 'right_barrels': 4, 'total...",0.07,0.14,[R],"{'R': {'FF': 62.359550561797754}, 'L': {'FF': ...",home,"[670770, 666158, 656582, 682829, 663886, 66871...",{},0,Reds,"25,749."
73,"Roupp, Landen",694738,777656,171,12,7.00,"{'left_barrels': 7, 'right_barrels': 1, 'total...",0.14,0.03,[L],"{'R': {'CU': 45.78313253012048, 'SI': 40.96385...",away,"[665487, 650333, 592518, 701538, 657757, 59342...","{'Sheets, Gavin': [0.14492753623188406, 0.4057...",2,Padres,6:45 PM.


In [90]:
# Barrel percent, blast percent
all_keys = {}
for index, row in hitter_pitcher_df.iterrows():
    for key in row['hitter_barrels']:
        all_keys[(row['hitter_team'], key)] = row['hitter_barrels'][key]
sorted_batters = all_keys.items()
sorted_batters = sorted(sorted_batters, key=lambda x: x[1], reverse=True)
for batter in sorted_batters:
    print(batter)

(('Dodgers', 'Ohtani, Shohei'), [np.float64(0.2839506172839506), np.float64(0.49382716049382713)])
(('Mariners', 'Raleigh, Cal'), [np.float64(0.24390243902439024), np.float64(0.4146341463414634)])
(('Blue Jays', 'Springer, George'), [np.float64(0.24193548387096775), np.float64(0.3387096774193548)])
(('Yankees', 'Judge, Aaron'), [np.float64(0.2318840579710145), np.float64(0.463768115942029)])
(('Marlins', 'Stowers, Kyle'), [np.float64(0.203125), np.float64(0.390625)])
(('Angels', 'Ward, Taylor'), [np.float64(0.20270270270270271), np.float64(0.28378378378378377)])
(('Cubs', 'Suzuki, Seiya'), [np.float64(0.18181818181818182), np.float64(0.4025974025974026)])
(('Dodgers', 'Conforto, Michael'), [np.float64(0.17647058823529413), np.float64(0.39215686274509803)])
(('Angels', 'Adell, Jo'), [np.float64(0.17647058823529413), np.float64(0.39215686274509803)])
(('Blue Jays', 'Bichette, Bo'), [np.float64(0.1724137931034483), np.float64(0.27586206896551724)])
(('Rockies', 'McMahon, Ryan'), [np.float

To do:
- Add ISO

In [91]:
len(sorted_batters)

36